# 基础理论部分

0. Can you come up out 3 sceneraies which use AI methods?

Ans: 语义分割、语音识别、语义推断
1. How do we use Github; Why do we use Jupyter and Pycharm;

Ans: Github便于存储、共享代码，相当于一个对程序员特化的网盘；Jupyter便于编译、演示代码和添加文字、公式说明；Pycharm比起Jupyter更善于用于大型项目，对各种包的管理也方便。
2. What's the Probability Model?

Ans:每个单词都有由频率为无偏估计的出现概率；句子拆为单词的时间序列后如果看作马         氏链可以有一个条件概率模型。
3. Can you came up with some sceneraies at which we could use Probability Model?

Ans:对一个特化场景的问答，如饭店应对顾客预约的电话，收集数据后可以很容易算出不         同的回答对某个提问的出现概率，选概率高的作为自动回答。
4. Why do we use probability and what's the difficult points for programming based on parsing and pattern match?

Ans:语义分析是本质的困难，机器没法像人一样综合利用经验、推断进行复杂的思考
5. What's the Language Model;

Ans:将句子看作词组按在句子中出现顺序的马氏链的概率模型。有了语料库后可以算出每         个句子的出现概率。
6. Can you came up with some sceneraies at which we could use Language Model?

Ans:同3.
7. What's the 1-gram language model;

Ans:每个词组是独立的，句子的概率等于所有词组的概率乘积。
8. What's the disadvantages and advantages of 1-gram language model;

Ans:优点是简单好算，缺点是没有实际效果，体现不出常见词组搭配，生成效果会很差。
9. What't the 2-gram models;

Ans:每个词组出现概率依赖前一个词组。假设sentence为w1,w2,...wN按顺序组成的，有
$$P(w_2|w_1)=\dfrac{P(w_2w_1)}{P(w_1)}$$
$$P(sentence)=\prod_{i=1}^{i=N-1}P(w_{i+1}|w_{i})P(w_1)$$.

# 编程实践部分

In [1]:
import numpy as np
import random
from jieba import cut
import re
import pandas as pd
from collections import Counter

In [2]:
human = """
human = 自己 寻找 活动
自己 = 我 | 俺 | 我们 
寻找 = 看看 | 找找 | 想找点
活动 = 乐子 | 玩的
"""
host = """
host = 寒暄 报数 询问 业务相关 结尾 
报数 = 我是 数字 号 ,
数字 = 单个数字 | 数字 单个数字 
单个数字 = 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 
寒暄 = 称谓 打招呼 | 打招呼
称谓 = 人称 ,
人称 = 先生 | 女士 | 小朋友
打招呼 = 你好 | 您好 
询问 = 请问你要 | 您需要
业务相关 = 玩玩 具体业务
玩玩 = 耍一耍 | 玩一玩
具体业务 = 喝酒 | 打牌 | 打猎 | 赌博
结尾 = 吗？"""

In [3]:
def create_grammar(gram,line_split='\n',split='='):
    #生成语法，这是一个python字典，之后可以用其树结构快速生成例句。树一般有三层，
    #由上往下依次是语法名、句子结构、用于填充每个结构的备选词集合。
    dicts={}
    for line in gram.split(line_split):
        if not line.strip():continue
        else:
            index,statement=line.split(split)
            dicts[index.strip()]=[s.split() for s in statement.split('|')]
            #这里要确保动作index(如host、报数、数字)的statement的list长度为1
            #但其他index的statement的list，每个状态贡献一个length
    return dicts
host_gram=create_grammar(host)
host_gram
human_gram=create_grammar(human)
human_gram

{'host': [['寒暄', '报数', '询问', '业务相关', '结尾']],
 '报数': [['我是', '数字', '号', ',']],
 '数字': [['单个数字'], ['数字', '单个数字']],
 '单个数字': [['1'], ['2'], ['3'], ['4'], ['5'], ['6'], ['7'], ['8'], ['9']],
 '寒暄': [['称谓', '打招呼'], ['打招呼']],
 '称谓': [['人称', ',']],
 '人称': [['先生'], ['女士'], ['小朋友']],
 '打招呼': [['你好'], ['您好']],
 '询问': [['请问你要'], ['您需要']],
 '业务相关': [['玩玩', '具体业务']],
 '玩玩': [['耍一耍'], ['玩一玩']],
 '具体业务': [['喝酒'], ['打牌'], ['打猎'], ['赌博']],
 '结尾': [['吗？']]}

{'human': [['自己', '寻找', '活动']],
 '自己': [['我'], ['俺'], ['我们']],
 '寻找': [['看看'], ['找找'], ['想找点']],
 '活动': [['乐子'], ['玩的']]}

In [4]:
def generator(gram,target):
    if target not in gram:
        return target
    else:
        expand = random.choice(gram[target])
        expand_recursion=[ generator(gram,t) for t in expand ]
        return ''.join(e for e in expand_recursion)
generator(host_gram,'host')

'小朋友,您好我是65号,您需要耍一耍打牌吗？'

In [5]:
def gernerator_n(gram,target,n=1):
    return [generator(gram,target) for i in range(n)]
gernerator_n(human_gram,'human',3)

['我们想找点玩的', '我们找找玩的', '我想找点玩的']

2. 使用新数据源完成语言模型的训练 \
按照我们上文中定义的prob_2函数，我们更换一个文本数据源，获得新的Language Model:\
下载文本数据集（你可以在以下数据集中任选一个，也可以两个都使用）\
可选数据集1，保险行业问询对话集： https://github.com/Computing-Intelligence/insuranceqa-corpus-zh/raw/release/corpus/pool/train.txt.gz \
可选数据集2：豆瓣评论数据集：https://github.com/Computing-Intelligence/datasource/raw/master/movie_comments.csv \
修改代码，获得新的2-gram语言模型 \
进行文本清洗，获得所有的纯文本 \
将这些文本进行切词 \
送入之前定义的语言模型中，判断文本的合理程度

In [6]:
#尝试了多种清洗数据的方式，步骤包含取出纯中文、去掉nan(用pandas可能涉及)
#第一种不好去掉数字和句子中英文
forshow_data_source=pd.read_csv('movie_comments.csv',low_memory=False)
forshow_data_source=forshow_data_source['comment'].tolist()
forshow_data=''
for i in forshow_data_source:
    if str(i) != 'nan':
        forshow_data=forshow_data+i
forshow_data=''.join(re.findall('\w+',forshow_data))
forshow_data[:200]

'吴京意淫到了脑残的地步看了恶心想吐首映礼看的太恐怖了这个电影不讲道理的完全就是吴京在实现他这个小粉红的英雄梦各种装备轮番上场视物理逻辑于不顾不得不说有钱真好随意胡闹吴京的炒作水平不输冯小刚但小刚至少不会用主旋律来炒作吴京让人看了不舒服为了主旋律而主旋律为了煽情而煽情让人觉得他是个大做作大谎言家729更新片子整体不如湄公河行动1整体不够流畅编剧有毒台词尴尬2刻意做作的主旋律煽情显得如此不合时宜而又多'

In [7]:
#第二种糙快猛，就用它了:
with open('movie_comments.csv') as f:
    L=f.readlines()
    pattern="[\u4e00-\u9fa5]+" 
    regex = re.compile(pattern)
    data1=[]
    for i in L:
        results =  regex.findall(i)
        data1+=results
data1=''.join(data1)
with open('train.txt') as f:
    L=f.readlines()
    pattern="[\u4e00-\u9fa5]+" 
    regex = re.compile(pattern)
    data2=[]
    for i in L:
        results =  regex.findall(i)
        data2+=results
data2=''.join(data2)
data=data1+data2
data[:200]

'战狼吴京意淫到了脑残的地步看了恶心想吐战狼首映礼看的太恐怖了这个电影不讲道理的完全就是吴京在实现他这个小粉红的英雄梦各种装备轮番上场视物理逻辑于不顾不得不说有钱真好随意胡闹战狼吴京的炒作水平不输冯小刚但小刚至少不会用主旋律来炒作吴京让人看了不舒服为了主旋律而主旋律为了煽情而煽情让人觉得他是个大做作大谎言家更新片子整体不如湄公河行动整体不够流畅编剧有毒台词尴尬刻意做作的主旋律煽情显得如此不合时宜而又'

In [8]:
data=list(cut(data))
token_2_gram=[''.join(data[i:i+2]) for i in range(len(data[:-2]))]
word_count_2=Counter(token_2_gram)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/7g/b3sd1r7d37j4ttzm2q7xpcwr0000gn/T/jieba.cache
Loading model cost 0.993 seconds.
Prefix dict has been built succesfully.


In [9]:
def prob_1(word):
    return words_count[word]/len(token_2_gram)
def prob_2(word1,word2):
    if word1+word2 in word_count_2:
        return word_count_2[word1+word2]/len(token_2_gram)
    else:
        return 1/len(token_2_gram)
def sentence_prob(sentence):
    words = list(cut(sentence))
    sentence_prob = 1
    for i in range(len(words)-1):
        word=words[i]
        next_word=words[i+1]
        probability=prob_2(word,next_word)
        sentence_prob*=probability
    return sentence_prob

3. 获得最优质的的语言
当我们能够生成随机的语言并且能判断之后，我们就可以生成更加合理的语言了。请定义 generate_best 函数，该函数输入一个语法 + 语言模型，能够生成n个句子，并能选择一个最合理的句子:


In [10]:
def generate_best(gram=host_gram,target='host',n=10):
    result=[]
    for i in range(n):
        sentence=generator(gram,target)
        result.append([sentence,sentence_prob(sentence)])
    return sorted(result,key=lambda x :x[1],reverse=True)
generate_best()

[['您好我是18号,您需要耍一耍喝酒吗？', 8.01200900411323e-77],
 ['你好我是69号,请问你要玩一玩打牌吗？', 9.150328392747928e-80],
 ['先生,你好我是2号,您需要耍一耍赌博吗？', 1.7869253168815048e-89],
 ['小朋友,您好我是21号,您需要耍一耍赌博吗？', 2.978208861469175e-90],
 ['先生,您好我是56号,您需要耍一耍喝酒吗？', 2.978208861469175e-90],
 ['先生,你好我是532593号,请问你要玩一玩喝酒吗？', 3.401342795626461e-93],
 ['小朋友,您好我是88号,请问你要玩一玩打猎吗？', 5.668904659377435e-94],
 ['先生,您好我是6号,请问你要玩一玩喝酒吗？', 5.668904659377435e-94],
 ['小朋友,你好我是3号,请问你要耍一耍打牌吗？', 1.1337809318754873e-94],
 ['先生,您好我是7号,请问你要耍一耍喝酒吗？', 1.889634886459145e-95]]